## Импорт библиотек

In [1]:
!pip install tensorflow

In [17]:
import gc

import os
import tqdm

import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


## Проверка среды

In [3]:
!nvidia-smi


Wed Jul  3 15:11:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.test.gpu_device_name()


Num GPUs Available:  1


'/device:GPU:0'

In [5]:
torch.cuda.is_available()


True

## Config

In [6]:
TRAIN_TRANSACTIONS_PATH = '/content/drive/My Drive/Университет/Credit_Scoring/data/train/'
TEST_TRANSACTIONS_PATH = '/content/drive/My Drive/Университет/Credit_Scoring/data/test/'
TRAIN_TARGET_PATH = '/content/drive/My Drive/Университет/Credit_Scoring/data/train_target.csv'
TEST_TARGET_PATH = '/content/drive/My Drive/Университет/Credit_Scoring/data/test_target_contest.csv'


## Загрузка данных

### Функции

In [7]:
import os
import pandas as pd
import tqdm

def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                     num_parts_to_read: int = 2, columns=None, verbose=False) -> pd.DataFrame:
    """
    читает num_parts_to_read партиций, преобразует их к pd.DataFrame и возвращает
    :param path_to_dataset: путь до директории с партициями
    :param start_from: номер партиции, с которой начать чтение
    :param num_parts_to_read: количество партиций, которые требуется прочитать
    :param columns: список колонок, которые нужно прочитать из партиции
    :return: pd.DataFrame
    """

    res = []
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset)
                              if filename.startswith('part')])  # Получаем список всех файлов, начинающихся с 'part'

    start_from = max(0, start_from)  # Убеждаемся, что стартовая партиция не меньше нуля
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]  # Выбираем нужное количество партиций для чтения
    if verbose:
        print('Reading chunks:\n')  # Если verbose=True, выводим на печать имена партиций
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса
        chunk = pd.read_parquet(chunk_path, columns=columns)  # Читаем текущую партицию, возможно, выбирая определенные колонки
        res.append(chunk)  # Добавляем прочитанную партицию в список
    return pd.concat(res).reset_index(drop=True)  # Объединяем все партиции в один DataFrame и сбрасываем индексы


### Импорт данных

In [8]:
data = read_parquet_dataset_from_local(TRAIN_TRANSACTIONS_PATH, start_from=0, num_parts_to_read=1)
data.info()


<ipython-input-7-a59935a93a70>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):  # Читаем партиции с отображением прогресса


Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5408648 entries, 0 to 5408647
Data columns (total 20 columns):
 #   Column                Dtype  
---  ------                -----  
 0   app_id                int32  
 1   amnt                  float64
 2   currency              int32  
 3   operation_kind        int32  
 4   card_type             int32  
 5   operation_type        int32  
 6   operation_type_group  int32  
 7   ecommerce_flag        int32  
 8   payment_system        int32  
 9   income_flag           int32  
 10  mcc                   int32  
 11  country               int32  
 12  city                  int32  
 13  mcc_category          int32  
 14  day_of_week           int32  
 15  hour                  int32  
 16  days_before           int32  
 17  weekofyear            int32  
 18  hour_diff             int64  
 19  transaction_number    int32  
dtypes: float64(1), int32(18), int64(1)
memory usage: 453.9 MB


In [9]:
target_df = pd.read_csv(TRAIN_TARGET_PATH)
target_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 963811 entries, 0 to 963810
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   app_id   963811 non-null  int64
 1   product  963811 non-null  int64
 2   flag     963811 non-null  int64
dtypes: int64(3)
memory usage: 22.1 MB


In [10]:
merged_data = data.merge(target_df[['app_id', 'product', 'flag']], on='app_id')
merged_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5408648 entries, 0 to 5408647
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   app_id                int32  
 1   amnt                  float64
 2   currency              int32  
 3   operation_kind        int32  
 4   card_type             int32  
 5   operation_type        int32  
 6   operation_type_group  int32  
 7   ecommerce_flag        int32  
 8   payment_system        int32  
 9   income_flag           int32  
 10  mcc                   int32  
 11  country               int32  
 12  city                  int32  
 13  mcc_category          int32  
 14  day_of_week           int32  
 15  hour                  int32  
 16  days_before           int32  
 17  weekofyear            int32  
 18  hour_diff             int64  
 19  transaction_number    int32  
 20  product               int64  
 21  flag                  int64  
dtypes: float64(1), int32(18), int64(3)
memory 

In [11]:
merged_data


,app_id,amnt,currency,operation_kind,card_type,operation_type,operation_type_group,ecommerce_flag,payment_system,income_flag,...,city,mcc_category,day_of_week,hour,days_before,weekofyear,hour_diff,transaction_number,product,flag
0,0,0.465425,1,4,98,4,2,3,7,3,...,37,2,4,19,351,34,-1,1,3,0
1,0,0.000000,1,2,98,7,1,3,7,3,...,49,2,4,20,351,34,0,2,3,0
2,0,0.521152,1,2,98,3,1,3,7,3,...,37,2,4,20,351,34,0,3,3,0
3,0,0.356078,1,1,5,2,1,3,7,3,...,49,7,2,0,348,34,52,4,3,0
4,0,0.000000,1,2,98,7,1,3,7,3,...,49,2,4,16,337,53,280,5,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5408643,23646,0.390944,1,1,37,2,1,1,3,1,...,3,1,6,9,2,48,32,453,1,0
5408644,23646,0.428447,1,1,37,2,1,1,3,1,...,120,7,5,12,1,48,27,454,1,0
5408645,23646,0.371478,1,1,37,2,1,1,3,1,...,2,9,5,13,1,48,1,455,1,0
5408646,23646,0.348726,1,1,37,2,1,1,3,1,...,3,9,5,13,1,48,1,456,1,0


In [12]:
features = [x for x in merged_data.columns if x not in ['app_id', 'flag']]
features


['amnt',
 'currency',
 'operation_kind',
 'card_type',
 'operation_type',
 'operation_type_group',
 'ecommerce_flag',
 'payment_system',
 'income_flag',
 'mcc',
 'country',
 'city',
 'mcc_category',
 'day_of_week',
 'hour',
 'days_before',
 'weekofyear',
 'hour_diff',
 'transaction_number',
 'product']

In [13]:
X = merged_data[features].values
y = merged_data['flag'].values

# Создание фолдов
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = list(skf.split(X, y))

# Проверка, сколько данных в каждом фолде
for fold, (train_idx, val_idx) in enumerate(folds):
    print(f"Fold {fold+1}:")
    print(f"  Train: {len(train_idx):_}")
    print(f"  Validation: {len(val_idx):_}")


Fold 1:
  Train: 4_326_918
  Validation: 1_081_730
Fold 2:
  Train: 4_326_918
  Validation: 1_081_730
Fold 3:
  Train: 4_326_918
  Validation: 1_081_730
Fold 4:
  Train: 4_326_919
  Validation: 1_081_729
Fold 5:
  Train: 4_326_919
  Validation: 1_081_729


## TensorFlow baseline

In [ ]:
def create_model(input_shape):
    # Инициализация последовательной модели
    model = Sequential()

    # Добавление слоя LSTM с 128 нейронами.
    # input_shape задает форму входных данных (длина последовательности, количество признаков).
    # return_sequences=True означает, что слой будет возвращать полные последовательности на каждом временном шаге.
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    # Добавление слоя Dropout с коэффициентом 0.2 для предотвращения переобучения.
    model.add(Dropout(0.2))

    # Добавление второго слоя LSTM с 64 нейронами.
    # return_sequences=False означает, что слой будет возвращать только последний выходной элемент.
    model.add(LSTM(64, return_sequences=False))
    # Добавление второго слоя Dropout с коэффициентом 0.2 для регуляризации.
    model.add(Dropout(0.2))

    # Добавление выходного слоя с одним нейроном и сигмоидной активацией.
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC'])

    return model


In [ ]:
# Ваши данные: X, y и folds
roc_auc_scores = []

# Обучение и оценка модели на каждом фолде
for fold, (train_idx, val_idx) in enumerate(folds):
    print(f"Training fold {fold+1}...")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Паддинг последовательностей для RNN
    # padding='post': Указывает, что паддинг (добавление нулей) будет происходить в конце каждой последовательности.
    X_train_padded = pad_sequences(X_train, padding='post', dtype='float32')
    X_val_padded = pad_sequences(X_val, padding='post', dtype='float32')

    # Добавление третьего измерения (количество признаков)
    # Функция np.expand_dims добавляет новое измерение в массив данных. Используется -1 как параметр для указания позиции,
    # куда добавить новое измерение, что в данном случае означает добавление его в конец.
    X_train_padded = np.expand_dims(X_train_padded, -1)
    X_val_padded = np.expand_dims(X_val_padded, -1)

    # Получение формы входных данных для создания модели
    # input_shape говорит модели, что она будет получать на вход
    # последовательности длиной 4 временных шага, и на каждом временном шаге будет один признак.
    input_shape = (X_train_padded.shape[1], X_train_padded.shape[2])

    with tf.device('/GPU:0'):
        model = create_model(input_shape)
        model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_data=(X_val_padded, y_val), verbose=2)

    y_val_pred = model.predict(X_val_padded).ravel()
    roc_auc = roc_auc_score(y_val, y_val_pred)
    roc_auc_scores.append(roc_auc)
    print(f"Fold {fold+1} ROC-AUC: {roc_auc:.4f}")

# Средний ROC-AUC по всем фолдам
mean_roc_auc = np.mean(roc_auc_scores)
print(f"Mean ROC-AUC: {mean_roc_auc:.4f}")

Training fold 1...
Epoch 1/5
67609/67609 - 490s - loss: 0.1093 - auc: 0.5838 - val_loss: 0.1069 - val_auc: 0.6399 - 490s/epoch - 7ms/step
Epoch 2/5
67609/67609 - 486s - loss: 0.1059 - auc: 0.6563 - val_loss: 0.1037 - val_auc: 0.6966 - 486s/epoch - 7ms/step
Epoch 3/5
67609/67609 - 484s - loss: 0.1022 - auc: 0.7048 - val_loss: 0.0995 - val_auc: 0.7350 - 484s/epoch - 7ms/step
Epoch 4/5
67609/67609 - 484s - loss: 0.0989 - auc: 0.7389 - val_loss: 0.0976 - val_auc: 0.7509 - 484s/epoch - 7ms/step
Epoch 5/5
67609/67609 - 482s - loss: 0.0964 - auc: 0.7611 - val_loss: 0.0939 - val_auc: 0.7793 - 482s/epoch - 7ms/step
33805/33805 [==============================] - 97s 3ms/step
Fold 1 ROC-AUC: 0.7817
Training fold 2...
Epoch 1/5


KeyboardInterrupt: 

-- прервал обучение после первого фолда--

Epoch 1/5: Это первая из пяти эпох обучения. В этой строке также указано, что всего будет проведено 5 эпох.


* 67609/67609: Количество батчей (шагов) на одну эпоху. Это количество раз, которое модель обновляет свои веса в течение одной эпохи.
* 490s: Общее время, затраченное на обучение модели на первой эпохе
* loss: 0.1093: Значение функции потерь на обучающем наборе данных. В данном случае используется бинарная кросс-энтропия. Более низкие значения функции потерь указывают на лучшую модель.
* auc: 0.5838: Значение метрики AUC (Area Under the Curve) на обучающем наборе данных. AUC измеряет способность модели различать классы. Значение 0.5 указывает на случайное угадывание, а 1.0 - на идеальную модель.
* val_loss: 0.1069: Значение функции потерь на проверочном (валидационном) наборе данных.
* val_auc: 0.6399: Значение метрики AUC на проверочном (валидационном) наборе данных.

## PyTorch baseline

In [16]:
# class LSTMModel(nn.Module):
#     def __init__(self, input_shape):
#         super(LSTMModel, self).__init__()
#         self.lstm1 = nn.LSTM(input_size=input_shape[1], hidden_size=64, batch_first=True)
#         self.dropout1 = nn.Dropout(0.2)
#         self.lstm2 = nn.LSTM(input_size=64, hidden_size=32, batch_first=True)
#         self.dropout2 = nn.Dropout(0.2)
#         self.fc = nn.Linear(32, 1)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         out, _ = self.lstm1(x)
#         out = self.dropout1(out)
#         out, _ = self.lstm2(out)
#         out = self.dropout2(out[:, -1, :])  # Берем только последний выход
#         out = self.fc(out)
#         out = self.sigmoid(out)
#         return out


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_shape):
        super(LSTMModel, self).__init__()
        # Создаем первый LSTM-слой
        # input_size=input_shape[1] указывает на количество признаков на каждом временном шаге
        # hidden_size=64 указывает на количество нейронов в скрытом слое LSTM
        # batch_first=True указывает, что первым измерением является размер батча
        self.lstm1 = nn.LSTM(input_size=input_shape[1], hidden_size=64, batch_first=True)

        # Добавляем слой Dropout с вероятностью 0.2 для предотвращения переобучения
        self.dropout1 = nn.Dropout(0.2)

        # Создаем второй LSTM-слой
        # input_size=64 указывает на количество признаков, выходящих из первого LSTM-слоя
        # hidden_size=32 указывает на количество нейронов во втором LSTM-слое
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=32, batch_first=True)

        # Добавляем второй слой Dropout с вероятностью 0.2 для предотвращения переобучения
        self.dropout2 = nn.Dropout(0.2)

        # Создаем полносвязный (линейный) слой
        # Входной размер 32 соответствует выходному размеру второго LSTM-слоя
        # Выходной размер 1 означает, что мы получаем один выход (вероятность)
        self.fc = nn.Linear(32, 1)

        # Добавляем сигмоидную активацию для получения вероятности (между 0 и 1)
        self.sigmoid = nn.Sigmoid()

In [18]:
roc_auc_scores = []

# Обучение и оценка модели на каждом фолде
for fold, (train_idx, val_idx) in enumerate(folds):
    print(f"Training fold {fold+1}...")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Паддинг последовательностей для RNN
    X_train_padded = torch.tensor(pad_sequences(X_train, padding='post', dtype=np.float32))
    X_val_padded = torch.tensor(pad_sequences(X_val, padding='post', dtype=np.float32))

    # Добавление третьего измерения (количество признаков)
    X_train_padded = X_train_padded.unsqueeze(-1)
    X_val_padded = X_val_padded.unsqueeze(-1)

    # Создание DataLoader для пакетной обработки данных
    train_dataset = TensorDataset(X_train_padded, torch.tensor(y_train, dtype=torch.float32))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    val_dataset = TensorDataset(X_val_padded, torch.tensor(y_val, dtype=torch.float32))
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Получение формы входных данных для создания модели
    input_shape = (X_train_padded.shape[1], X_train_padded.shape[2])

    # Инициализация модели и оптимизатора
    model = LSTMModel(input_shape)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())

    # Переключение на использование GPU, если доступно
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Обучение модели
    model.train()
    for epoch in range(5):
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs.squeeze(), batch_y)
            loss.backward()
            optimizer.step()

    # Оценка модели на валидационном наборе
    model.eval()
    y_val_pred = []
    with torch.no_grad():
        for batch_x, _ in val_loader:
            batch_x = batch_x.to(device)
            outputs = model(batch_x)
            y_val_pred.extend(outputs.cpu().detach().numpy().ravel())

    roc_auc = roc_auc_score(y_val, y_val_pred)
    roc_auc_scores.append(roc_auc)
    print(f"Fold {fold+1} ROC-AUC: {roc_auc:.4f}")

# Средний ROC-AUC по всем фолдам
mean_roc_auc = np.mean(roc_auc_scores)
print(f"Mean ROC-AUC: {mean_roc_auc:.4f}")


Training fold 1...
Fold 1 ROC-AUC: 0.6947
Training fold 2...


KeyboardInterrupt: 